In [4]:
import sys
sys.path.append("../")

import requests
import random
import json
from tqdm.notebook import tqdm
import pandas as pd
from metrics.WebResource import WebResource

In [7]:
def get_live_deploys_urls():
    results = []
    live_deploys_remote_file = "https://raw.githubusercontent.com/BioSchemas/bioschemas.github.io/master/_data/live_deployments.json"
    res = requests.get(live_deploys_remote_file)
    live_deploys = res.json()
    for r in random.sample(live_deploys["resources"], 5):
    #for r in live_deploys["resources"]:
        results.append(r["url"])
    return results

def test_ld_bioschemas_annot():
        res = get_live_deploys_urls()
        errors = []
        results = []
        
        i = 0
        for r in tqdm(res):
        #for r in reversed(res):
            row = {}
            status_code = requests.head(r).status_code
            print(r)
            row['url'] = r
            print("Status code: " + str(status_code))
            row['status code'] = str(status_code)
            try :
                kg = WebResource(r).get_rdf()
                print("Triples: " + str(len(kg)))
                row['Retrieved triples'] = len(kg)
                if not len(kg) > 0:
                    print(f"# Error with {r}")
                    row['Error'] = True
                    errors.append(r)
            except UnicodeDecodeError as err:
                print(err)
            results.append(row)

        print(f"{len(res)} tested URLS")
        print(f"{len(errors)} failing URLS")
        print(errors)
        df = pd.read_json(json.dumps(results))
        return df

In [8]:
df = test_ld_bioschemas_annot()
df

  0%|          | 0/5 [00:00<?, ?it/s]

https://modelarchive.org/
Status code: 200
8
{'@context': 'file:///Users/gaignard-a/Documents/Dev/fair-checker/static/data/jsonldcontext.json', 'description': 'ModelArchive provides a stable accession code (DOI) for deposited theoretical models of protein and macromolecular structures', 'name': 'ModelArchive', '@type': 'DataCatalog', 'url': 'https://www.modelarchive.org/', 'keywords': 'model, archive, ModelArchive, homology model, protein structure models', 'license': {'@type': 'CreativeWork', 'name': 'Creative Commons CC4 Attribution', 'url': 'https://creativecommons.org/licenses/by/4.0/'}, 'publisher': [{'@type': 'Organization', 'name': 'SIB Swiss Institute of Bioinformatics', 'url': 'https://sib.swiss'}, {'@type': 'Organization', 'name': 'Biozentrum - University of Basel', 'url': 'https://www.biozentrum.unibas.ch'}]}
<class 'dict'>
Triples: 34
https://www.omicsdi.org/
Status code: 200
1
{'@graph': [{'name': 'Omics Discovery Index - Discovering and Linking Public Omics Datasets', 'ur

ERROR:DEV:Cannot parse RDF from https://www.omicsdi.org/ due to UnicodeDecodeError
ERROR:DEV:'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte
ERROR:DEV:Cannot parse RDF from https://www.omicsdi.org/ due to UnicodeDecodeError
ERROR:DEV:'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte


Cannot parse RDF from https://www.omicsdi.org/ due to UnicodeDecodeError
Triples: 0
# Error with https://www.omicsdi.org/
http://gigadb.org/
Status code: 200
Triples: 14
https://deb-central.org/
Status code: 405
Expecting value: line 1 column 1 (char 0)
Triples: 0
# Error with https://deb-central.org/
https://galaxyproject.org/
Status code: 200


Triples: 0
# Error with https://galaxyproject.org/
5 tested URLS
3 failing URLS
['https://www.omicsdi.org/', 'https://deb-central.org/', 'https://galaxyproject.org/']


,url,status code,Retrieved triples,Error
0,https://modelarchive.org/,200,34,NaN
1,https://www.omicsdi.org/,200,0,1.0
2,http://gigadb.org/,200,14,NaN
3,https://deb-central.org/,405,0,1.0
4,https://galaxyproject.org/,200,0,1.0
